<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# TL;DR

1. In this lab scenario you will have a chance to compare performance of the classic RNN and LSTM on a toy example. 
2. This toy example will show that maintaining memory over even 20 steps is non-trivial. 
3. Finally, you will see how curriculum learning may allow to train a model on larger sequences.

# Problem definition

Here we consider a toy example, where the goal is to discriminate between two types of binary sequences:
* [Type 0] a sequence with exactly one zero (remaining entries are equal to one).
* [Type 1] a sequence full of ones,

We are especially interested in the performance of the trained models on discriminating between a sequence full of ones versus a sequence with leading zero followed by ones. Note that in this case the goal of the model is to output the first element of the sequence, as the label (sequence type) is fully determined by the first element of the sequence.

#Implementation

## Importing torch

Install `torch` and `torchvision`

In [ ]:
!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

## Understand dimensionality

Check the input and output specification [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) and [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). The following snippet shows how we can process
a sequence by LSTM and output a vector of size `hidden_dim` after reading
each token of the sequence. 

In [ ]:
hidden_dim = 5
lstm = nn.LSTM(1, hidden_dim)  # Input sequence contains elements - vectors of size 1

# create a random sequence
sequence = [torch.randn(1) for _ in range(10)]

In [ ]:
# initialize the hidden state (including cell state)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

for i, elem in enumerate(sequence):
  # we are processing only a single element of the sequence, and there
  # is only one sample (sequence) in the batch, the third one
  # corresponds to the fact that our sequence contains elemenents,
  # which can be treated as vectors of size 1
  out, hidden = lstm(elem.view(1, 1, 1), hidden)
  print(f'i={i} out={out.detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

i=0 out=tensor([[[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]]])
i=1 out=tensor([[[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]]])
i=2 out=tensor([[[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]]])
i=3 out=tensor([[[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]]])
i=4 out=tensor([[[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]]])
i=5 out=tensor([[[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]]])
i=6 out=tensor([[[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]]])
i=7 out=tensor([[[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]]])
i=8 out=tensor([[[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]]])
i=9 out=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]])
Final hidden state=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]]) cell state=tensor([[[-0.2590,  0.4080,  0.1307,  0.4329, -0.2895]]])


## To implement

Process the whole sequence all at once by calling `lstm` only once and check that the output is exactly the same as above (remember to initialize the hidden state the same way).

In [ ]:
# #########################################################
#                    To implement
# #########################################################

# make the sequence a tensor of correct size
sequence = torch.tensor(sequence).view(10, 1, 1)

# initialize the hidden state (including cell state)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

output, (final_hidden_state, final_cell_state) = lstm(sequence, hidden)
for i, elem in enumerate(output):
  print(f'i={i} out={elem.detach()}')
print(f'Final hidden state={final_hidden_state.detach()} cell state={final_cell_state.detach()}')

i=0 out=tensor([[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]])
i=1 out=tensor([[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]])
i=2 out=tensor([[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]])
i=3 out=tensor([[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]])
i=4 out=tensor([[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]])
i=5 out=tensor([[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]])
i=6 out=tensor([[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]])
i=7 out=tensor([[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]])
i=8 out=tensor([[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]])
i=9 out=tensor([[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]])
Final hidden state=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]]) cell state=tensor([[[-0.2590,  0.4080,  0.1307,  0.4329, -0.2895]]])


## Training a model

Below we define a very simple model, which is a single layer of LSTM, where the output in each time step is processed by relu followed by a single fully connected layer, the output of which is a single number. We are going
to use the number generated after reading the last element of the sequence,
which will serve as the logit for our classification problem.

In [ ]:
class Model(nn.Module):

    def __init__(self, hidden_dim):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

Below is a training loop, where we only train on the two hardest examples.

In [ ]:
SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]


def eval_on_hard_examples(model):
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')


def train_model(hidden_dim, lr, num_steps=10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_hard_examples(model)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

In [ ]:
train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([-0.1775]), tensor([-0.1774])]
Logits for hard examples=[tensor([0.0095]), tensor([0.0097])]
Logits for hard examples=[tensor([0.0259]), tensor([0.0263])]
Logits for hard examples=[tensor([-0.0017]), tensor([-0.0009])]
Logits for hard examples=[tensor([-0.0034]), tensor([7.1838e-05])]
Logits for hard examples=[tensor([-3.0632]), tensor([3.5531])]
Logits for hard examples=[tensor([-11.0094]), tensor([10.8645])]
Logits for hard examples=[tensor([-12.1567]), tensor([11.9191])]
Logits for hard examples=[tensor([-12.3136]), tensor([12.0644])]
Logits for hard examples=[tensor([-12.3368]), tensor([12.0874])]
Logits for hard examples=[tensor([-12.3420]), tensor([12.0939])]
Logits for hard examples=[tensor([-12.3448]), tensor([12.0982])]
Logits for hard examples=[tensor([-12.3473]), tensor([12.1022])]
Logits for hard examples=[tensor([-12.3498]), tensor([12.1062])]
Logits for hard examples=[tensor([-12.3523]), tensor([12.1101])]
Logits for hard examples=[tensor(

## To implement

1. Check for what values of `SEQUENCE_LEN` the model is able to discriminate between the two hard examples (after training).

Note that for steps 2-4 you may need to change the value of `num_steps`.


In [ ]:
SEQUENCE_LEN = 11

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([0.0854]), tensor([0.0853])]
Logits for hard examples=[tensor([0.0340]), tensor([0.0340])]
Logits for hard examples=[tensor([-0.0007]), tensor([-0.0007])]
Logits for hard examples=[tensor([-0.0031]), tensor([-0.0031])]
Logits for hard examples=[tensor([0.0018]), tensor([0.0018])]
Logits for hard examples=[tensor([0.0021]), tensor([0.0021])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0014])]
Logits for hard examples=[tensor([0.0024]), tensor([0.0025])]
Logits for hard examples=[tensor([0.0015]), tensor([0.0016])]
Logits for hard examples=[tensor([0.0014]), tensor([0.0015])]
Logits for hard examples=[tensor([0.0016]), tensor([0.0019])]
Logits for hard examples=[tensor([0.0015]), tensor([0.0020])]
Logits for hard examples=[tensor([0.0004]), tensor([0.0017])]
Logits for hard examples=[tensor([-0.0101]), tensor([0.0129])]
Logits for hard examples=[tensor([-9.9209]), tensor([10.1397])]
Logits for hard examples=[tensor([-13.2050]), tensor([12.8995])

In [ ]:
SEQUENCE_LEN = 12

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([-0.0323]), tensor([-0.0323])]
Logits for hard examples=[tensor([0.0250]), tensor([0.0249])]
Logits for hard examples=[tensor([-0.0012]), tensor([-0.0012])]
Logits for hard examples=[tensor([-0.0006]), tensor([-0.0006])]
Logits for hard examples=[tensor([0.0026]), tensor([0.0026])]
Logits for hard examples=[tensor([0.0011]), tensor([0.0011])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0014])]
Logits for hard examples=[tensor([0.0014]), tensor([0.0014])]
Logits for hard examples=[tensor([0.0015]), tensor([0.0015])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0014])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0014])]
Logits for hard examples=[tensor([1.7098e-05]), tensor([8.5090e-05])]
Logits for hard examples=[tensor([0.0016]), tensor([0.00

In [ ]:
SEQUENCE_LEN = 13

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([-0.1013]), tensor([-0.1013])]
Logits for hard examples=[tensor([-0.0147]), tensor([-0.0147])]
Logits for hard examples=[tensor([0.0120]), tensor([0.0121])]
Logits for hard examples=[tensor([0.0062]), tensor([0.0064])]
Logits for hard examples=[tensor([0.0001]), tensor([0.0005])]
Logits for hard examples=[tensor([0.0005]), tensor([0.0013])]
Logits for hard examples=[tensor([-7.7274e-05]), tensor([0.0053])]
Logits for hard examples=[tensor([-8.3054]), tensor([7.3554])]
Logits for hard examples=[tensor([-14.7091]), tensor([12.6479])]
Logits for hard examples=[tensor([-15.5905]), tensor([13.3639])]
Logits for hard examples=[tensor([-15.7088]), tensor([13.4607])]
Logits for hard examples=[tensor([-15.7247]), tensor([13.4746])]
Logits for hard examples=[tensor([-15.7268]), tensor([13.4774])]
Logits for hard examples=[tensor([-15.7271]), tensor([13.4787])]
Logits for hard examples=[tensor([-15.7271]), tensor([13.4798])]
Logits for hard examples=[tensor([-15.7

In [ ]:
SEQUENCE_LEN = 14

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([-0.0441]), tensor([-0.0441])]
Logits for hard examples=[tensor([0.0232]), tensor([0.0232])]
Logits for hard examples=[tensor([0.0038]), tensor([0.0038])]
Logits for hard examples=[tensor([-0.0020]), tensor([-0.0020])]
Logits for hard examples=[tensor([0.0023]), tensor([0.0023])]
Logits for hard examples=[tensor([0.0012]), tensor([0.0012])]
Logits for hard examples=[tensor([0.0012]), tensor([0.0012])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0012]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logi

In [ ]:
SEQUENCE_LEN = 15

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([-0.1748]), tensor([-0.1748])]
Logits for hard examples=[tensor([0.0550]), tensor([0.0550])]
Logits for hard examples=[tensor([-0.0008]), tensor([-0.0008])]
Logits for hard examples=[tensor([-0.0043]), tensor([-0.0043])]
Logits for hard examples=[tensor([0.0046]), tensor([0.0046])]
Logits for hard examples=[tensor([0.0002]), tensor([0.0002])]
Logits for hard examples=[tensor([0.0014]), tensor([0.0014])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0012]), tensor([0.0012])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Lo

The model is able to discriminate between the two hard examples for sequences of length smaller or equal to 13 and fails to do so on loges sequences.


2. Instead of training on `HARD_EXAMPLES` only, modify the training loop to train on sequences where zero may be in any position of the sequence (so any valid sequence of `Type 0`, not just the hardest one). After modifying the training loop check for what values of `SEQUENCE_LEN` you can train the model successfully.

In [ ]:
import random 

SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

# Returns a sequence of ones or ones with a zero on a random index based on given label.
def get_normal_sequence(label):
    sequence = SEQUENCE_LEN*[1.]
    if (label == 0):
        sequence[random.randint(0, SEQUENCE_LEN-1)] = 0.
    return sequence

def eval_on_normal_examples(model):
    with torch.no_grad():
        logits = []
        for _, label in HARD_EXAMPLES:
            sequence = get_normal_sequence(label)
            input = torch.tensor(sequence).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for normal examples={logits}')


def train_model_on_normal_examples(hidden_dim, lr, num_steps=10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_normal_examples(model)

        for _, label in HARD_EXAMPLES:
            sequence = get_normal_sequence(label)
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

In [ ]:
SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model_on_normal_examples(hidden_dim=20, lr=0.01, num_steps=2000)

Logits for normal examples=[tensor([0.3403]), tensor([0.3471])]
Logits for normal examples=[tensor([-0.8592]), tensor([0.3931])]
Logits for normal examples=[tensor([0.0774]), tensor([0.0778])]
Logits for normal examples=[tensor([-0.1452]), tensor([0.2099])]
Logits for normal examples=[tensor([1.4544]), tensor([1.4544])]
Logits for normal examples=[tensor([0.2659]), tensor([0.2659])]
Logits for normal examples=[tensor([-0.9263]), tensor([-0.2137])]
Logits for normal examples=[tensor([0.2077]), tensor([0.1392])]
Logits for normal examples=[tensor([0.1637]), tensor([0.1637])]
Logits for normal examples=[tensor([0.2536]), tensor([0.2536])]
Logits for normal examples=[tensor([0.0455]), tensor([0.0455])]
Logits for normal examples=[tensor([-0.0239]), tensor([-0.0240])]
Logits for normal examples=[tensor([0.0174]), tensor([0.0174])]
Logits for normal examples=[tensor([0.0183]), tensor([0.0183])]
Logits for normal examples=[tensor([0.0125]), tensor([0.0125])]
Logits for normal examples=[tensor

In [ ]:
SEQUENCE_LEN = 20

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model_on_normal_examples(hidden_dim=20, lr=0.01, num_steps=2000)

Logits for normal examples=[tensor([-0.1796]), tensor([-0.1797])]
Logits for normal examples=[tensor([0.0294]), tensor([0.0294])]
Logits for normal examples=[tensor([-0.0512]), tensor([-0.0189])]
Logits for normal examples=[tensor([0.0871]), tensor([0.0872])]
Logits for normal examples=[tensor([0.1617]), tensor([0.1619])]
Logits for normal examples=[tensor([0.0730]), tensor([0.0752])]
Logits for normal examples=[tensor([0.3456]), tensor([0.3465])]
Logits for normal examples=[tensor([0.0022]), tensor([0.0077])]
Logits for normal examples=[tensor([-0.0541]), tensor([-0.0541])]
Logits for normal examples=[tensor([-0.0686]), tensor([-0.0550])]
Logits for normal examples=[tensor([-0.0969]), tensor([0.0520])]
Logits for normal examples=[tensor([0.0629]), tensor([0.0621])]
Logits for normal examples=[tensor([0.1110]), tensor([0.1111])]
Logits for normal examples=[tensor([-0.0666]), tensor([-0.0665])]
Logits for normal examples=[tensor([0.0287]), tensor([0.0289])]
Logits for normal examples=[t

In [ ]:
SEQUENCE_LEN = 25

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model_on_normal_examples(hidden_dim=20, lr=0.01, num_steps=2000)

Logits for normal examples=[tensor([0.1276]), tensor([0.1277])]
Logits for normal examples=[tensor([0.0346]), tensor([0.0360])]
Logits for normal examples=[tensor([-0.0794]), tensor([-0.0718])]
Logits for normal examples=[tensor([-0.0179]), tensor([-0.0168])]
Logits for normal examples=[tensor([-0.0425]), tensor([-0.0193])]
Logits for normal examples=[tensor([0.0101]), tensor([0.0102])]
Logits for normal examples=[tensor([0.1903]), tensor([0.1955])]
Logits for normal examples=[tensor([-0.0935]), tensor([-0.0935])]
Logits for normal examples=[tensor([-0.5117]), tensor([-0.5116])]
Logits for normal examples=[tensor([0.1132]), tensor([0.1136])]
Logits for normal examples=[tensor([-0.2766]), tensor([-0.2756])]
Logits for normal examples=[tensor([-0.0781]), tensor([-0.0083])]
Logits for normal examples=[tensor([-0.2832]), tensor([-0.2831])]
Logits for normal examples=[tensor([-0.0745]), tensor([-0.0745])]
Logits for normal examples=[tensor([0.1123]), tensor([0.1130])]
Logits for normal exam

In [ ]:
SEQUENCE_LEN = 26

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model_on_normal_examples(hidden_dim=20, lr=0.01, num_steps=2000)

Logits for normal examples=[tensor([-0.2013]), tensor([-0.2013])]
Logits for normal examples=[tensor([0.0386]), tensor([0.0399])]
Logits for normal examples=[tensor([-0.0189]), tensor([0.0342])]
Logits for normal examples=[tensor([0.0349]), tensor([0.0351])]
Logits for normal examples=[tensor([-0.1620]), tensor([-0.1398])]
Logits for normal examples=[tensor([0.0205]), tensor([0.0211])]
Logits for normal examples=[tensor([-0.5050]), tensor([-0.5050])]
Logits for normal examples=[tensor([-0.1914]), tensor([-0.1914])]
Logits for normal examples=[tensor([-0.0689]), tensor([-0.0674])]
Logits for normal examples=[tensor([-0.0098]), tensor([-0.0098])]
Logits for normal examples=[tensor([0.0893]), tensor([0.0898])]
Logits for normal examples=[tensor([0.2192]), tensor([0.2198])]
Logits for normal examples=[tensor([-0.1116]), tensor([0.1624])]
Logits for normal examples=[tensor([0.2898]), tensor([0.2901])]
Logits for normal examples=[tensor([-0.0883]), tensor([-0.0879])]
Logits for normal exampl

When trained on normal sentences, the model is able to discriminate between two hard examples for sequences of length <= 25 (for num_steps=2000).


3. Replace LSTM by a classic RNN and check for what values of `SEQUENCE_LEN` you can train the model successfully.

In [ ]:
class Model(nn.Module):

    def __init__(self, hidden_dim):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

In [ ]:
import random 

SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

def eval_on_normal_examples(model):
    with torch.no_grad():
        logits = []
        for _, label in HARD_EXAMPLES:
            sequence = get_normal_sequence(label)
            input = torch.tensor(sequence).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for normal examples={logits}')


def train_model_on_normal_examples(hidden_dim, lr, num_steps=10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_normal_examples(model)

        for _, label in HARD_EXAMPLES:
            sequence = get_normal_sequence(label)
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

def eval_on_hard_examples(model):
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')


def train_model_on_hard_examples(hidden_dim, lr, num_steps=10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_hard_examples(model)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

In [ ]:
SEQUENCE_LEN = 5

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model_on_normal_examples(hidden_dim=20, lr=0.01, num_steps=2000)

Logits for normal examples=[tensor([0.0534]), tensor([0.0551])]
Logits for normal examples=[tensor([0.2178]), tensor([-0.0652])]
Logits for normal examples=[tensor([-4.6292]), tensor([3.8117])]
Logits for normal examples=[tensor([-8.3657]), tensor([6.0672])]
Logits for normal examples=[tensor([-7.2535]), tensor([7.6421])]
Logits for normal examples=[tensor([-8.4040]), tensor([7.7039])]
Logits for normal examples=[tensor([-17.1719]), tensor([7.8308])]
Logits for normal examples=[tensor([-9.5871]), tensor([7.9181])]
Logits for normal examples=[tensor([-17.2917]), tensor([8.0080])]
Logits for normal examples=[tensor([-10.0237]), tensor([8.0818])]
Logits for normal examples=[tensor([-14.3475]), tensor([8.1574])]
Logits for normal examples=[tensor([-14.3589]), tensor([8.2222])]
Logits for normal examples=[tensor([-14.3401]), tensor([8.3094])]
Logits for normal examples=[tensor([-17.4374]), tensor([8.3915])]
Logits for normal examples=[tensor([-14.3059]), tensor([8.4444])]
Logits for normal 

In [ ]:
SEQUENCE_LEN = 7

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model_on_normal_examples(hidden_dim=20, lr=0.01, num_steps=2000)

Logits for normal examples=[tensor([0.0710]), tensor([0.0687])]
Logits for normal examples=[tensor([0.4350]), tensor([0.5277])]
Logits for normal examples=[tensor([-0.1034]), tensor([-0.1034])]
Logits for normal examples=[tensor([0.2924]), tensor([0.2924])]
Logits for normal examples=[tensor([-2.5613]), tensor([-2.5123])]
Logits for normal examples=[tensor([-0.0704]), tensor([-0.0703])]
Logits for normal examples=[tensor([0.3971]), tensor([0.3971])]
Logits for normal examples=[tensor([-1.6011]), tensor([1.9049])]
Logits for normal examples=[tensor([0.5605]), tensor([0.5605])]
Logits for normal examples=[tensor([-0.0688]), tensor([-0.0688])]
Logits for normal examples=[tensor([0.0633]), tensor([0.0633])]
Logits for normal examples=[tensor([-0.0113]), tensor([-0.0113])]
Logits for normal examples=[tensor([0.0279]), tensor([0.0279])]
Logits for normal examples=[tensor([0.0096]), tensor([0.0096])]
Logits for normal examples=[tensor([0.0175]), tensor([0.0175])]
Logits for normal examples=[t

In [ ]:
SEQUENCE_LEN = 8

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

train_model_on_normal_examples(hidden_dim=20, lr=0.01, num_steps=2000)

Logits for normal examples=[tensor([0.0592]), tensor([0.0532])]
Logits for normal examples=[tensor([0.2696]), tensor([0.2381])]
Logits for normal examples=[tensor([-0.6512]), tensor([0.0539])]
Logits for normal examples=[tensor([-0.1600]), tensor([-0.1600])]
Logits for normal examples=[tensor([0.1757]), tensor([0.1757])]
Logits for normal examples=[tensor([0.2338]), tensor([0.2338])]
Logits for normal examples=[tensor([-0.2404]), tensor([-0.2404])]
Logits for normal examples=[tensor([-0.0387]), tensor([-0.0387])]
Logits for normal examples=[tensor([0.0368]), tensor([0.0368])]
Logits for normal examples=[tensor([0.0225]), tensor([0.0225])]
Logits for normal examples=[tensor([0.0103]), tensor([0.0103])]
Logits for normal examples=[tensor([0.0371]), tensor([0.0334])]
Logits for normal examples=[tensor([1.1129]), tensor([1.1129])]
Logits for normal examples=[tensor([-4.6684]), tensor([0.7637])]
Logits for normal examples=[tensor([-5.1905]), tensor([0.5639])]
Logits for normal examples=[ten

Our RNN model works good for hard sequences of length <= 7 (for num_steps=2000).


4. Write a proper curricullum learning loop, where in a loop you consider longer and longer sequences, where expansion of the sequence length happens only after the model is trained successfully on the current length.

In [ ]:
import random 

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

# Returns a sequence of ones or ones with a zero on a random index based on given label.
def get_normal_sequence_of_given_length(label, length):
    sequence = length*[1.]
    if (label == 0):
        sequence[random.randint(0, length-1)] = 0.
    return sequence

def eval_on_normal_examples(model, length):
    with torch.no_grad():
        logits = []
        for _, label in HARD_EXAMPLES:
            sequence = get_normal_sequence_of_given_length(label, length)
            input = torch.tensor(sequence).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for normal examples={logits}, score={abs(logits[1].item() - logits[0].item())}')
        return abs(logits[1].item() - logits[0].item()) > 5


def train_model_on_normal_examples(length, hidden_dim, lr, num_steps=10000, max_length=20, min_times=100):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)
    times = 1

    for step in range(num_steps):  
      if step % 100 == 0:
        if eval_on_normal_examples(model, length):
          times += 1
          if times == min_times:
            times = 1
            length += 1
            print(f'The length is now {length}')
            if length == max_length:
              break

      for _, label in HARD_EXAMPLES:
        sequence = get_normal_sequence_of_given_length(label, length)
        model.zero_grad()
        logit = model(torch.tensor(sequence).view(-1, 1, 1))  
              
        loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
        loss.backward()

        optimizer.step()   

In [ ]:
SEQUENCE_LEN = 2

train_model_on_normal_examples(SEQUENCE_LEN, hidden_dim=20, lr=0.01, num_steps=10000, min_times=10)

Logits for normal examples=[tensor([0.0174]), tensor([-0.0386])], score=0.056044748052954674
Logits for normal examples=[tensor([-6.2102]), tensor([8.2213])], score=14.431459426879883
Logits for normal examples=[tensor([-8.4256]), tensor([11.2166])], score=19.642135620117188
Logits for normal examples=[tensor([-12.8428]), tensor([11.5330])], score=24.375752449035645
Logits for normal examples=[tensor([-8.8468]), tensor([11.6134])], score=20.460264205932617
Logits for normal examples=[tensor([-8.8820]), tensor([11.6574])], score=20.539387702941895
Logits for normal examples=[tensor([-8.9113]), tensor([11.6930])], score=20.604293823242188
Logits for normal examples=[tensor([-8.9393]), tensor([11.7243])], score=20.66359519958496
Logits for normal examples=[tensor([-13.0060]), tensor([11.7498])], score=24.755763053894043
Logits for normal examples=[tensor([-13.0232]), tensor([11.7806])], score=24.803720474243164
The length is now 3
Logits for normal examples=[tensor([-8.8884]), tensor([11.